In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os



In [2]:
df_final = pd.read_pickle('results/df_final.pkl')
df_final.shape

(204942, 58)

In [3]:
df_final.columns

Index(['timestamp', 'device', 'ping_ms', 'datarate', 'jitter', 'Latitude',
       'Longitude', 'Altitude', 'speed_kmh', 'COG', 'precipIntensity',
       'precipProbability', 'temperature', 'humidity', 'windSpeed',
       'Traffic Jam Factor', 'Traffic Street Name', 'Traffic Distance',
       'Pos in Ref Round', 'measurement', 'area', 'PCell_RSRP_1',
       'PCell_RSRP_2', 'PCell_RSRP_max', 'PCell_RSRQ_1', 'PCell_RSRQ_2',
       'PCell_RSRQ_max', 'PCell_RSSI_1', 'PCell_RSSI_2', 'PCell_RSSI_max',
       'PCell_SNR_1', 'PCell_SNR_2', 'PCell_E-ARFCN', 'PCell_Downlink_Num_RBs',
       'PCell_Downlink_TB_Size', 'PCell_Downlink_Average_MCS',
       'PCell_Uplink_Num_RBs', 'PCell_Uplink_TB_Size',
       'PCell_Uplink_Tx_Power_(dBm)', 'PCell_Cell_ID',
       'PCell_Downlink_frequency', 'PCell_Uplink_frequency',
       'PCell_Downlink_bandwidth_MHz', 'PCell_Uplink_bandwidth_MHz',
       'PCell_Cell_Identity', 'PCell_TAC', 'PCell_Band_Indicator',
       'PCell_freq_MHz', 'scenario', 'drive_mode',

In [4]:

# Calculate missing percentage for all features
missing_stats = pd.DataFrame({
    'feature': df_final.columns,
    'missing_count': df_final.isnull().sum().values,
    'missing_pct': (df_final.isnull().sum() / len(df_final) * 100).values
}).sort_values('missing_pct', ascending=False)

# Categorize features by missing percentage
print(f"\nMISSING DATA:")

categories = {
    'Complete (0%)': (0, 0),
    'Low (0-5%)': (0.01, 5),
    'Informative (5-50%)': (5.01, 50),
    'High (50-80%)': (50.01, 80),
    'Very High (80-95%)': (80.01, 95),
    'Almost Complete (95-100%)': (95.01, 100)
}

for cat_name, (lower, upper) in categories.items():
    if lower == 0 and upper == 0:
        mask = missing_stats['missing_pct'] == 0
    else:
        mask = (missing_stats['missing_pct'] >= lower) & (missing_stats['missing_pct'] <= upper)
    
    count = mask.sum()
    print(f"   {cat_name:30s}: {count:>3d} features")


MISSING DATA:
   Complete (0%)                 :  24 features
   Low (0-5%)                    :  15 features
   Informative (5-50%)           :  18 features
   High (50-80%)                 :   0 features
   Very High (80-95%)            :   1 features
   Almost Complete (95-100%)     :   0 features


In [6]:
# Category 1: Informative range (5-50% missing) - Create individual indicators
informative_features = missing_stats[
    (missing_stats['missing_pct'] >= 5) & 
    (missing_stats['missing_pct'] <= 50)
]['feature'].tolist()

print(f"\nINFORMATIVE RANGE (5-50% missing): {len(informative_features)} features")

for feat in informative_features:
    missing_pct = missing_stats[missing_stats['feature'] == feat]['missing_pct'].values[0]
    print(f"     {feat:50s} {missing_pct:>6.2f}%")



INFORMATIVE RANGE (5-50% missing): 18 features
     ping_ms                                             28.16%
     datarate                                             7.65%
     jitter                                               7.65%
     Pos in Ref Round                                     6.91%
     PCell_Uplink_frequency                               6.15%
     PCell_Band_Indicator                                 6.15%
     PCell_Downlink_frequency                             6.15%
     PCell_Cell_ID                                        6.15%
     PCell_Uplink_bandwidth_MHz                           6.15%
     PCell_Downlink_bandwidth_MHz                         6.15%
     PCell_Cell_Identity                                  6.15%
     PCell_TAC                                            6.15%
     PCell_Uplink_Tx_Power_(dBm)                          5.96%
     PCell_Uplink_TB_Size                                 5.96%
     PCell_Uplink_Num_RBs                               

In [7]:
# Category 2: Informative range (50-80% missing) - Create individual indicators
other_features = missing_stats[
    (missing_stats['missing_pct'] >= 50) & 
    (missing_stats['missing_pct'] <= 80)
]['feature'].tolist()

print(f"Other RANGE (50-80% missing): {len(other_features)} features")

for feat in other_features:
    missing_pct = missing_stats[missing_stats['feature'] == feat]['missing_pct'].values[0]
    print(f"   • {feat:50s} {missing_pct:>6.2f}%")


Other RANGE (50-80% missing): 0 features


In [8]:


# Very High (80-95%)
very_high = missing_stats[(missing_stats['missing_pct'] > 80) & (missing_stats['missing_pct'] <= 95)]
print(f"\nVERY HIGH (80-95%): {len(very_high)} features")

pcell_very_high = very_high[very_high['feature'].str.startswith('PCell')]
scell_very_high = very_high[very_high['feature'].str.startswith('SCell')]

print(f"    PCell: {len(pcell_very_high)} features")
print(f"    SCell: {len(scell_very_high)} features")

# Almost complete (95-100%)
almost_complete = missing_stats[(missing_stats['missing_pct'] > 95) & (missing_stats['missing_pct'] < 100)]
print(f"\nALMOST COMPLETE (95-100%): {len(almost_complete)} features")

pcell_almost = almost_complete[almost_complete['feature'].str.startswith('PCell')]
scell_almost = almost_complete[almost_complete['feature'].str.startswith('SCell')]

print(f"    PCell: {len(pcell_almost)} features")
print(f"    SCell: {len(scell_almost)} features")



VERY HIGH (80-95%): 1 features
    PCell: 0 features
    SCell: 0 features

ALMOST COMPLETE (95-100%): 0 features
    PCell: 0 features
    SCell: 0 features


In [9]:

# Low missing (0-5%)
low_missing = missing_stats[(missing_stats['missing_pct'] > 0) & (missing_stats['missing_pct'] <= 5)]
print(f"\nLOW MISSING (0-5%): {len(low_missing)} features")
print(f"{'Feature':<50s} {'Missing %':>10s} {'Type'}")

for idx, row in low_missing.iterrows():
    feat = row['feature']
    pct = row['missing_pct']
    
  
    
    print(f"{feat:<50s} {pct:>9.2f}% ")


LOW MISSING (0-5%): 15 features
Feature                                             Missing % Type
PCell_freq_MHz                                          4.89% 
PCell_SNR_2                                             4.89% 
PCell_E-ARFCN                                           4.89% 
PCell_RSRQ_max                                          4.89% 
PCell_SNR_1                                             4.89% 
PCell_RSSI_2                                            4.89% 
PCell_RSSI_1                                            4.89% 
PCell_RSSI_max                                          4.89% 
PCell_RSRQ_2                                            4.89% 
PCell_RSRQ_1                                            4.89% 
PCell_RSRP_max                                          4.89% 
PCell_RSRP_2                                            4.89% 
PCell_RSRP_1                                            4.89% 
Traffic Jam Factor                                      1.40% 
Altitude          

In [10]:
df_engineered = df_final.copy()

In [11]:

# Identify PCell features in 50-95% missing range
pcell_high_features = missing_stats[
    (missing_stats['missing_pct'] > 50) & 
    (missing_stats['missing_pct'] <= 95) &
    (missing_stats['feature'].str.startswith('PCell'))
]['feature'].tolist()


# Create group indicator
if len(pcell_high_features) > 0:
    df_engineered['pcell_high_missing'] = df_final[pcell_high_features].isnull().all(axis=1).astype(int)
    
    pcell_high_dist = df_engineered['pcell_high_missing'].value_counts()
    print(f"\nCreated: pcell_high_missing")
    print(f"   Total Feature: {len(pcell_high_features)} PCell features")
    print(f"   Distribution:")
    print(f"     Present (0): {pcell_high_dist.get(0, 0):>7,} rows ({pcell_high_dist.get(0, 0)/len(df_engineered)*100:>5.2f}%)")
    print(f"     Missing (1): {pcell_high_dist.get(1, 0):>7,} rows ({pcell_high_dist.get(1, 0)/len(df_engineered)*100:>5.2f}%)")
    
    print(f"\n Created 1 PCell high group indicator")
else:
    print("\n No PCell features in 50-95% range")


 No PCell features in 50-95% range


In [14]:
all_indicator_cols = [col for col in df_engineered.columns if col.endswith('_missing')]

print(f"\nCREATED INDICATORS:")
print(f"   Informative (5-50%):        {len(informative_features):>3d} indicators")
print(f"   SCell group:                  1 indicator")
print(f"   PCell high group:             1 indicator")
print(f"   TOTAL:                      {len(all_indicator_cols):>3d} indicators")

# Verify no missing values in indicators
indicator_missing = df_engineered[all_indicator_cols].isnull().sum().sum()
if indicator_missing == 0:
    print(f"\n All {len(all_indicator_cols)} indicators are complete (no missing values)")
else:
    print(f"\n Warning: {indicator_missing} missing values in indicators")

print(f"\nCurrent dataset shape: {df_engineered.shape}")




CREATED INDICATORS:
   Informative (5-50%):         18 indicators
   SCell group:                  1 indicator
   PCell high group:             1 indicator
   TOTAL:                        0 indicators

 All 0 indicators are complete (no missing values)

Current dataset shape: (204942, 58)


In [15]:
categorical_cols = df_engineered.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"{'Feature':<30s} {'Unique Values':>15s} {'Type':<15s}")

for col in categorical_cols:
    n_unique = df_engineered[col].nunique()
    dtype = str(df_engineered[col].dtype)
    print(f"{col:<30s} {n_unique:>15d} {dtype:<15s}")

Feature                          Unique Values Type           
device                                       4 object         
Traffic Street Name                         64 object         
area                                         6 object         
PCell_Downlink_bandwidth_MHz                 4 object         
PCell_Uplink_bandwidth_MHz                   4 object         
scenario                                     4 object         
drive_mode                                   3 object         
direction                                    2 object         
measured_qos                                 2 object         


In [16]:


# Categorize by cardinality
low_cardinality = []
high_cardinality = []

for col in categorical_cols:
    n_unique = df_engineered[col].nunique()
    if n_unique <= 10:
        low_cardinality.append(col)
    else:
        high_cardinality.append(col)

print(f"\nLOW CARDINALITY (≤10 unique): {len(low_cardinality)} features")
for col in low_cardinality:
    n_unique = df_engineered[col].nunique()
    values = df_engineered[col].value_counts()
    print(f"\n   {col} ({n_unique} unique values):")
    for val, count in values.items():
        print(f"      {val}: {count:,} ({count/len(df_engineered)*100:.2f}%)")

print(f"\nHIGH CARDINALITY : {len(high_cardinality)} features")
for col in high_cardinality:
    n_unique = df_engineered[col].nunique()
    print(f"    {col}: {n_unique} unique values → Consider dropping")


LOW CARDINALITY (≤10 unique): 8 features

   device (4 unique values):
      pc3: 59,723 (29.14%)
      pc1: 59,519 (29.04%)
      pc2: 42,879 (20.92%)
      pc4: 42,821 (20.89%)

   area (6 unique values):
      Park: 73,409 (35.82%)
      Residential: 62,746 (30.62%)
      Avenue: 43,466 (21.21%)
      Highway: 23,482 (11.46%)
      Tunnel: 1,097 (0.54%)
      UNKNOWN: 742 (0.36%)

   PCell_Downlink_bandwidth_MHz (4 unique values):
      20 : 155,911 (76.08%)
      15 : 35,651 (17.40%)
      5 : 712 (0.35%)
      10 : 61 (0.03%)

   PCell_Uplink_bandwidth_MHz (4 unique values):
      20 : 155,911 (76.08%)
      15 : 35,651 (17.40%)
      5 : 712 (0.35%)
      10 : 61 (0.03%)

   scenario (4 unique values):
      A3D: 64,626 (31.53%)
      A3U: 63,355 (30.91%)
      A2U: 42,730 (20.85%)
      A2D: 34,231 (16.70%)

   drive_mode (3 unique values):
      platoon: 158,579 (77.38%)
      1x2: 27,747 (13.54%)
      2x2: 18,616 (9.08%)

   direction (2 unique values):
      uplink: 106,085

In [17]:
low_cardinality

['device',
 'area',
 'PCell_Downlink_bandwidth_MHz',
 'PCell_Uplink_bandwidth_MHz',
 'scenario',
 'drive_mode',
 'direction',
 'measured_qos']

In [18]:
#convert to int to float
bandwidth_cols=['PCell_Downlink_bandwidth_MHz',
 'PCell_Uplink_bandwidth_MHz',
 'SCell_Downlink_bandwidth_MHz',
 'SCell_Uplink_bandwidth_MHz']


for col in bandwidth_cols:
    if col in df_engineered.columns:
        df_engineered[col] = pd.to_numeric(df_engineered[col], errors='coerce')


In [19]:
if len(high_cardinality) > 0:
    for col in high_cardinality:
        n_unique = df_engineered[col].nunique()
        print(f" {col} ({n_unique} unique) → Too many categories")
    
    df_engineered = df_engineered.drop(columns=high_cardinality)
    print(f"Dropped {len(high_cardinality)} features")

 Traffic Street Name (64 unique) → Too many categories
Dropped 1 features


In [20]:
low_cad = ['device']

# One-hot encode
df_engineered = pd.get_dummies(
    df_engineered, 
    columns=low_cad, 
    dtype=int
)

# Drop a specific dummy column
df_engineered = df_engineered.drop(columns=['device_pc4'], errors='ignore')

In [21]:

low_cal=[
 'area',
 'scenario',
 'drive_mode',
 'direction',
 'measured_qos']
# One-hot encode low cardinality features

print(f"\n ONE-HOT ENCODING LOW CARDINALITY:")

# Get features to encode (exclude timestamp, date)
features_to_encode = [col for col in low_cal 
                        if col not in ['timestamp', 'ts_gps', 'date']]

print(f"  Encoding {len(features_to_encode)} features with drop_first=True...")

# Store original shape
shape_before = df_engineered.shape

# One-hot encode
df_engineered = pd.get_dummies(df_engineered, 
                                columns=features_to_encode, 
                                drop_first=True,
                                dtype=int)

shape_after = df_engineered.shape



 ONE-HOT ENCODING LOW CARDINALITY:
  Encoding 5 features with drop_first=True...


In [22]:


print(f"  • Before: {shape_before[1]} features")
print(f"  • After:  {shape_after[1]} features")
print(f"  • Added:  {shape_after[1] - shape_before[1]} dummy variables")

# Show created dummy columns
new_cols = [col for col in df_engineered.columns if any(f'{feat}_' in col for feat in features_to_encode)]
print(f"\n  Created dummy variables ({len(new_cols)}):")


  • Before: 59 features
  • After:  66 features
  • Added:  7 dummy variables

  Created dummy variables (12):


In [23]:
new_cols

['area_Highway',
 'area_Park',
 'area_Residential',
 'area_Tunnel',
 'area_UNKNOWN',
 'scenario_A2U',
 'scenario_A3D',
 'scenario_A3U',
 'drive_mode_2x2',
 'drive_mode_platoon',
 'direction_uplink',
 'measured_qos_delay']

In [24]:
# Binary encode 'operator' (1,2 → 0,1)
if 'operator' in df_engineered.columns:
    df_engineered['operator'] = df_engineered['operator'] - 1



In [25]:


# Check timestamp column
timestamp_col = 'timestamp'
if timestamp_col not in df_engineered.columns:
    print("Error: 'timestamp' column not found")
else:
    print(f" Found timestamp column: {timestamp_col}")
    print(f"  • Data type: {df_engineered[timestamp_col].dtype}")
    print(f"  • Date range: {df_engineered[timestamp_col].min()} to {df_engineered[timestamp_col].max()}")
    
    # Ensure datetime type
    if not pd.api.types.is_datetime64_any_dtype(df_engineered[timestamp_col]):
        print("\n  Converting to datetime...")
        df_engineered[timestamp_col] = pd.to_datetime(df_engineered[timestamp_col])
        print(f"Converted to: {df_engineered[timestamp_col].dtype}")
    
    # Extract temporal features
    print("\n Extracting temporal features...")
    
    # Hour of day (0-23)
    df_engineered['hour'] = df_engineered[timestamp_col].dt.hour
    print(f"   hour: {df_engineered['hour'].min()}-{df_engineered['hour'].max()} range")
    
    # Day of week (0=Monday, 6=Sunday)
    df_engineered['day_of_week'] = df_engineered[timestamp_col].dt.dayofweek
    print(f"   day_of_week: {df_engineered['day_of_week'].min()}-{df_engineered['day_of_week'].max()} range")



 Found timestamp column: timestamp
  • Data type: datetime64[us, Europe/Berlin]
  • Date range: 2021-06-22 09:49:54+02:00 to 2021-06-24 18:59:59+02:00

 Extracting temporal features...
   hour: 8-18 range
   day_of_week: 1-3 range


In [26]:


print(f"\n  Hour distribution:")
hour_dist = df_engineered['hour'].value_counts().sort_index()
for hour, count in hour_dist.items():
    print(f"    {hour:02d}:00 - {count:>7,} rows ({count/len(df_engineered)*100:>5.2f}%)")

print(f"\n  Day of week distribution:")
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_dist = df_engineered['day_of_week'].value_counts().sort_index()
for dow, count in dow_dist.items():
    bar = '█' * int(count / len(df_engineered) * 50)
    print(f"    {dow} ({day_names[dow]:9s}) - {count:>7,} rows ({count/len(df_engineered)*100:>5.2f}%)")


print(f"\n Current dataset shape: {df_engineered.shape}")


  Hour distribution:
    08:00 -   1,096 rows ( 0.53%)
    09:00 -  21,904 rows (10.69%)
    10:00 -  31,036 rows (15.14%)
    11:00 -  29,842 rows (14.56%)
    12:00 -  11,509 rows ( 5.62%)
    13:00 -   2,248 rows ( 1.10%)
    14:00 -  19,454 rows ( 9.49%)
    15:00 -  27,001 rows (13.17%)
    16:00 -  32,927 rows (16.07%)
    17:00 -  19,143 rows ( 9.34%)
    18:00 -   8,782 rows ( 4.29%)

  Day of week distribution:
    1 (Tuesday  ) -  79,564 rows (38.82%)
    2 (Wednesday) -  79,015 rows (38.55%)
    3 (Thursday ) -  46,363 rows (22.62%)

 Current dataset shape: (204942, 68)


In [27]:
# Check if device and operator dummy columns exist
device_cols = [col for col in df_engineered.columns if col.startswith('device_')]
operator_cols = [col for col in df_engineered.columns if col.startswith('operator')]

print(f"   Device dummies:   {len(device_cols)}")
print(f"   Operator dummies: {len(operator_cols)}")

   Device dummies:   3
   Operator dummies: 1


In [28]:
device_switch = df_engineered[df_engineered['device_pc2'] == 1]['operator'].nunique() > 1
device_switch

False

In [29]:
device_data = df_engineered[df_engineered['device_pc1'] == 1]

unique_operators =device_data['operator'].value_counts()
unique_operators

operator
0    59519
Name: count, dtype: int64

In [30]:
direction_cols = [col for col in df_engineered.columns if col.startswith('direction_')]
scenario_cols = [col for col in df_engineered.columns if col.startswith('scenario_')]

print(f"   Direction dummies:   {len(direction_cols)}")
print(f"   Scenario dummies:    {len(scenario_cols)}")

   Direction dummies:   1
   Scenario dummies:    3


In [31]:


interaction_cols = [col for col in df_engineered.columns if '_X_' in col]

print(f"\nTOTAL INTERACTION FEATURES: {len(interaction_cols)}")
print(f"   Device × Operator:      {len([c for c in interaction_cols if 'device_' in c and 'operator' in c])}")
print(f"   Direction × Scenario:   {len([c for c in interaction_cols if 'direction_' in c and 'scenario_' in c])}")

print(f"\nCurrent dataset shape: {df_engineered.shape}")




TOTAL INTERACTION FEATURES: 0
   Device × Operator:      0
   Direction × Scenario:   0

Current dataset shape: (204942, 68)


In [32]:
df_engineered.shape

(204942, 68)

In [33]:

# Check if we have PCell_RSRP_max
if 'PCell_RSRP_max' in df_engineered.columns:
    # RSRP threshold: < -100 dBm is poor signal
    # Create indicator (1 = poor signal OR missing)
    
    rsrp_values = df_engineered['PCell_RSRP_max'].dropna()
    
    if len(rsrp_values) > 0:
        threshold = -100  # dBm
        
        df_engineered['poor_signal_quality'] = (
            (df_engineered['PCell_RSRP_max'] < threshold) | 
            (df_engineered['PCell_RSRP_max'].isnull())
        ).astype(int)
        
        poor_signal_dist = df_engineered['poor_signal_quality'].value_counts()
        poor_signal_count = poor_signal_dist.get(1, 0)
        
        print(f"\n Created: poor_signal_quality (threshold: {threshold} dBm)")
        print(f"   Good signal (0):  {poor_signal_dist.get(0, 0):>7,} rows ({poor_signal_dist.get(0, 0)/len(df_engineered)*100:>5.2f}%)")
        print(f"   Poor signal (1):  {poor_signal_count:>7,} rows ({poor_signal_count/len(df_engineered)*100:>5.2f}%)")
        
        # Show RSRP distribution
        print(f"\n  RSRP distribution:")
        print(f"      Min:    {rsrp_values.min():.2f} dBm")
        print(f"      25%:    {rsrp_values.quantile(0.25):.2f} dBm")
        print(f"      Median: {rsrp_values.median():.2f} dBm")
        print(f"      75%:    {rsrp_values.quantile(0.75):.2f} dBm")
        print(f"      Max:    {rsrp_values.max():.2f} dBm")
        print(f"\n  Meaning: 1 = Signal below {threshold} dBm (poor coverage)")
    else:
        print("\n  Cannot create poor_signal_quality (no valid RSRP values)")
else:
    print("\n  Cannot create poor_signal_quality (PCell_RSRP_max not found)")



 Created: poor_signal_quality (threshold: -100 dBm)
   Good signal (0):  155,216 rows (75.74%)
   Poor signal (1):   49,726 rows (24.26%)

  RSRP distribution:
      Min:    -176.70 dBm
      25%:    -98.26 dBm
      Median: -90.53 dBm
      75%:    -81.88 dBm
      Max:    -54.66 dBm

  Meaning: 1 = Signal below -100 dBm (poor coverage)


In [34]:
df_engineered['poor_signal_quality']

216       0
217       0
218       0
219       0
220       0
         ..
207429    0
207430    0
207431    0
207432    0
207433    0
Name: poor_signal_quality, Length: 204942, dtype: int64

In [35]:
df_engineered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204942 entries, 216 to 207433
Data columns (total 69 columns):
 #   Column                        Non-Null Count   Dtype                        
---  ------                        --------------   -----                        
 0   timestamp                     204942 non-null  datetime64[us, Europe/Berlin]
 1   ping_ms                       147235 non-null  float64                      
 2   datarate                      189263 non-null  float64                      
 3   jitter                        189263 non-null  float64                      
 4   Latitude                      204942 non-null  float64                      
 5   Longitude                     204942 non-null  float64                      
 6   Altitude                      204851 non-null  float64                      
 7   speed_kmh                     204942 non-null  float64                      
 8   COG                           204942 non-null  float64             

In [36]:
# Save as pickle (preserves dtypes, faster)
df_engineered.to_pickle('results/df_engineered.pkl')
print("\n Saved: results/df_engineered.pkl")

# Save as CSV 
df_engineered.to_csv('results/df_engineered.csv', index=False)
print(" Saved: results/df_engineered.csv")

df_engineered.to_parquet('results/df_engineered.parquet')


 Saved: results/df_engineered.pkl
 Saved: results/df_engineered.csv


In [37]:
df_corr=pd.read_csv('results/high_correlation_pairs.csv')
df_corr

,Feature_1,Feature_2,Correlation,Abs_Correlation
0,PCell_Downlink_frequency,PCell_Uplink_frequency,1.000000,1.000000
1,SCell_Downlink_RBs_MCS_2,SCell_Downlink_RBs_MCS_17,1.000000,1.000000
2,SCell_Downlink_RBs_MCS_10,SCell_Downlink_RBs_MCS_17,1.000000,1.000000
3,SCell_RSSI_2,SCell_RSSI_max,1.000000,1.000000
4,SCell_Downlink_RBs_MCS_17,SCell_Downlink_RBs_MCS_27,1.000000,1.000000
...,...,...,...,...
149,PCell_Downlink_RBs_MCS_14,PCell_Downlink_RBs_MCS_15,0.702839,0.702839
150,PCell_Downlink_RBs_MCS_13,PCell_Downlink_RBs_MCS_14,0.701664,0.701664
151,PCell_Downlink_Num_RBs,PCell_Downlink_RBs_MCS_30,0.701104,0.701104
152,PCell_Downlink_RBs_MCS_18,PCell_Downlink_RBs_MCS_19,0.700751,0.700751
